# Mini Sudoku
## What is mini-sudoku?
Well, it is Sudoku, but mini. Instead of 3x3 blocks of 3x3 hosting integer numbers from 1 to 9, the mini-sudoku has 3x2 blocks of 2x3 hosting integer from 1 to 6.

Here is an example:

![Mini-Sudoku](./images/Mini-Sudoku.png)

## Importing packages and setting constants

In [1]:
from dataclasses import dataclass
from ortools.sat.python.cp_model import CpModel, CpSolver, IntVar, OPTIMAL
from tabulate import tabulate

## A Class to manage each block

In [2]:
class SudokuBlock:
    BLOCK_ROW_SIZE: int = 2
    BLOCK_COL_SIZE: int = 3
    def __init__(
        self,
        model: CpModel,
        block_row_id: int,
        block_col_id: int,
        block_row_size: int = BLOCK_ROW_SIZE,
        block_col_size: int = BLOCK_COL_SIZE,
        verbose: bool = False
    ):
        self._verbose = verbose
        
        self._model = model
        
        self._block_row_id = block_row_id
        self._block_col_id = block_col_id
        self._block_name = f"b_{block_row_id}_{block_col_id}"
        self._block_row_size = block_row_size
        self._block_col_size = block_col_size
        self._block_size = block_row_size * block_col_size
        
        self._block_model_variables: list[IntVar] = []
        self.init_model()
        
    @property
    def name(self) -> str:
        return self._block_name
        
    def init_model(self):
        if self._verbose:
            print(f"Initializing block variables for block ({self._block_row_id},{self._block_col_id}) ...")
        self._block_model_variables = [
            self._model.NewIntVar(
                lb=1,
                ub=self._block_size,
                name=f"{self.name}_{cell_block_row_id}_{cell_block_col_id}"
            )    
            for cell_block_row_id in range(self._block_row_size)
            for cell_block_col_id in range(self._block_col_size)
        ]

        # All Different Values
        if self._verbose:
            print("Enforcing All different values within a block ...")
        self._model.AddAllDifferent(self._block_model_variables)
        
    def block_cell_row_col_to_idx(self, block_cell_row_id: int, block_cell_col_id: int):
        if (block_cell_row_id < 0) or (block_cell_row_id >= self._block_row_size):
            raise ValueError(f"block_cell_row_id ({block_cell_row_id}) must be between 1 and {self._block_row_size - 1}.")

        if (block_cell_col_id < 0) or (block_cell_col_id >= self._block_col_size):
            raise ValueError(f"block_cell_col_id ({block_cell_col_id}) must be between 1 and {self._block_col_size - 1}.")

        return block_cell_row_id * self._block_col_size + block_cell_col_id

    def get_block_cell(self, block_cell_row_id: int, block_cell_col_id: int):
        return self._block_model_variables[self.block_cell_row_col_to_idx(block_cell_row_id, block_cell_col_id)]

## Mini Sudoku Solver

In [3]:
class MiniSudokuSolver:
    GRID_ROW_SIZE: int = 3
    GRID_COL_SIZE: int = 2
    def __init__(
        self,
        init_values: list[tuple[int, int, int]],
        grid_row_size: int = GRID_ROW_SIZE,
        grid_col_size: int = GRID_COL_SIZE,
        block_row_size: int = SudokuBlock.BLOCK_ROW_SIZE,
        block_col_size: int = SudokuBlock.BLOCK_COL_SIZE,
        verbose: bool = False
    ):
        self._verbose = verbose
        self._model: CpModel = CpModel()
        self._solver: CpSolver = CpSolver()

        self._grid_row_size=grid_row_size
        self._grid_col_size=grid_col_size
        self._block_row_size=block_row_size
        self._block_col_size=block_col_size

        self._board_row_size = self._grid_row_size * self._block_row_size
        self._board_col_size = self._grid_col_size * self._block_col_size

        self._board_blocks: list[SudokuBlock] = []

        self.init_board(init_values)

    @property
    def row_range(self) -> range:
        return range(self._board_row_size)

    @property
    def col_range(self) -> range:
        return range(self._board_col_size)

    def get_block(self, block_row_id: int, block_col_id: int) -> SudokuBlock:
        if (block_row_id < 0) or (block_row_id >= self._grid_row_size):
            raise ValueError(f"block_row_id ({block_row_id}) must be between 0 and {self._grid_row_size - 1}.")

        if (block_col_id < 0) or (block_col_id >= self._grid_row_size):
            raise ValueError(f"block_col_id ({block_col_id}) must be between 0 and {self._grid_col_size - 1}.")

        block_id = block_row_id * self._grid_col_size + block_col_id
        return self._board_blocks[block_id]
            
    def get_cell(self, row_id: int, col_id: int):
        if (row_id < 0) or (row_id >= self._board_row_size):
            raise ValueError(f"row_id ({row_id}) must be between 0 and {self._board_row_size - 1}.")

        if (row_id < 0) or (row_id >= self._board_col_size):
            raise ValueError(f"row_id ({row_id}) must be between 0 and {self._board_col_size - 1}.")

        block_row_id: int = row_id // self._block_row_size
        block_col_id: int = col_id // self._block_col_size

        cell_block_row_id: int = row_id % self._block_row_size
        cell_block_col_id: int = col_id % self._block_col_size

        return self.get_block(block_row_id, block_col_id).get_block_cell(cell_block_row_id, cell_block_col_id)

    def get_value(self, row_id, col_id) -> int:
        return self._solver.value(
            self.get_cell(row_id, col_id)
        )

    def init_board(self, init_values: list[tuple[int, int, int]]):
        if self._verbose:
            print("Initializing board blocks ...")
        self._board_blocks = [
            SudokuBlock(
                model=self._model,
                block_row_id=block_row_id,
                block_col_id=block_col_id,
                block_row_size=self._block_row_size,
                block_col_size=self._block_col_size,
                verbose=self._verbose
            )
            for block_row_id in range(self._grid_row_size)
            for block_col_id in range(self._grid_col_size)
        ]

        # All Different each row
        if self._verbose:
            print("Enforcing All different values in a row ...")
        for row_id in self.row_range:
            self._model.AddAllDifferent([
                self.get_cell(row_id, col_id)
                for col_id in self.col_range
            ])

        # All Different each column
        if self._verbose:
            print("Enforcing All different values in a column ...")
        for col_id in self.col_range:
            self._model.AddAllDifferent([
                self.get_cell(row_id, col_id)
                for row_id in self.row_range
            ])

        # Setting initial values for cells
        if self._verbose:
            print("Setting Initial Cell Values ...")
        for row_id, col_id, value in init_values:
            self._model.add(
                self.get_cell(row_id, col_id) == value
            )

        if self._verbose:
            print("Ready to solve.")

    def solve(self, print_solution: bool = False) -> bool:
        status = self._solver.solve(self._model)

        output = status == OPTIMAL
        if output and print_solution:
            solution = [
                [
                    self.get_value(row_id, col_id)
                    for col_id in self.col_range
                ]
                for row_id in self.row_range
            ]

            print(tabulate(solution, tablefmt="grid"))

        return output

## Trying it out

### 2025-11-15

In [4]:
mini_sudoku_solver = MiniSudokuSolver(
    init_values=[
        (0, 0, 1),
        (1, 0, 2),
        (1, 5, 6),
        (2, 1, 3),
        (2, 5, 5),
        (3, 1, 4),
        (3, 4, 3),
        (4, 4, 4),
    ]
)
if mini_sudoku_solver.solve(print_solution=True):
    print("Found a solution.")
else:
    print("Failed to find a solution.")

+---+---+---+---+---+---+
| 1 | 6 | 3 | 2 | 5 | 4 |
+---+---+---+---+---+---+
| 2 | 5 | 4 | 3 | 1 | 6 |
+---+---+---+---+---+---+
| 6 | 3 | 1 | 4 | 2 | 5 |
+---+---+---+---+---+---+
| 5 | 4 | 2 | 6 | 3 | 1 |
+---+---+---+---+---+---+
| 3 | 1 | 6 | 5 | 4 | 2 |
+---+---+---+---+---+---+
| 4 | 2 | 5 | 1 | 6 | 3 |
+---+---+---+---+---+---+
Found a solution.


### 2025-11-16

In [ ]:
mini_sudoku_solver = MiniSudokuSolver(
    init_values=[
        (0, 0, 1),
        (1, 0, 2),
        (1, 5, 6),
        (2, 1, 3),
        (2, 5, 5),
        (3, 1, 4),
        (3, 4, 3),
        (4, 4, 4),
    ]
)
if mini_sudoku_solver.solve(print_solution=True):
    print("Found a solution.")
else:
    print("Failed to find a solution.")